In [86]:
#! pip install pandas-datareader
#! pip install pmdarima
#! pip install plotly
import numpy as np
from pandas_datareader import DataReader # pip install pandas-datareader
from pandas_datareader import data
from datetime import datetime
from pmdarima.arima import *
from pmdarima import preprocessing

from statsmodels import api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

import pandas as pd
import pmdarima as pm
import plotly.graph_objects as go
import plotly.express as px
 
start = pd.to_datetime('2010-01-01') # in YYYY-MM-DD format
end = pd.to_datetime('2018-01-01')
ts = data.DataReader('NDAQ', 'yahoo', start , end) # here 'yahoo' is the API to yahoo

In [44]:
fig = px.line(ts.Close, x=[ts.index], y="Close")
fig.show()

# Seasonal Values

# Seasonal Tests:

In [72]:
def seasonal_tests(data):
    result = pm.arima.nsdiffs(data, m=12, max_D=12, test='ch')
    
    print("CH results: " + str(result))
    
    result = pm.arima.nsdiffs(data, m=12, max_D=12, test='ocsb')
    
    print("OCSB results: " + str(result))


C:\Users\rache\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Lag Period:

In [87]:
def find_lag_period(data):
    
    adf_test = ADFTest(alpha = 0.05)

    decomp_results = seasonal_decompose(data, period=7)
    
    week_result = adf_test.should_diff(x=decomp_results.resid)

    decomp_results = seasonal_decompose(data, period=14)
    
    fortnight_result = adf_test.should_diff(x=decomp_results.resid)

    decomp_results = seasonal_decompose(data, period=30)
    
    month_result = adf_test.should_diff(x=decomp_results.resid)

    decomp_results = seasonal_decompose(data, period=120)
    
    season_result = adf_test.should_diff(x=decomp_results.resid)

    decomp_results = seasonal_decompose(data, period=365)
    
    year_result = adf_test.should_diff(x=decomp_results.resid)
    
    if (week_result < fortnight_result & week_result < month_result & week_result < season_result & week_result < year_result):
        return 7;
    if (fortnight_result < week_result & fortnight_result < month_result & fortnight_result < season_result & fortnight_result < year_result ):
        return 14;
    if (month_result < week_result & month_result < fortnight_result & month_result < season_result & month_result < year_result):
        return 30;
    if (season_result < week_result & season_result < fortnight_result & season_result < month_result & season_result < year_result):
        return 120;
    if (year_result < week_result & year_result < fortnight_result & year_result < season_result & year_result < month_result):
        return 365;

C:\Users\rache\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [88]:
find_lag_period(ts.Close)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [66]:
def sarima_model (data):
#train_test_split
    stepwise_model_7 = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    
    return stepwise_model_7

In [71]:
model_7 = sarima_model(ts.Close)

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[7]             : AIC=inf, Time=5.36 sec
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=4795.849, Time=0.07 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=4275.548, Time=0.23 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=5.65 sec
 ARIMA(1,1,0)(0,1,0)[7]             : AIC=4793.468, Time=0.10 sec
 ARIMA(1,1,0)(2,1,0)[7]             : AIC=4009.461, Time=0.58 sec
 ARIMA(1,1,0)(2,1,1)[7]             : AIC=inf, Time=5.16 sec
 ARIMA(1,1,0)(1,1,1)[7]             : AIC=inf, Time=2.59 sec
 ARIMA(0,1,0)(2,1,0)[7]             : AIC=4013.685, Time=0.33 sec
 ARIMA(2,1,0)(2,1,0)[7]             : AIC=4008.358, Time=0.61 sec
 ARIMA(2,1,0)(1,1,0)[7]             : AIC=4274.611, Time=0.43 sec
 ARIMA(2,1,0)(2,1,1)[7]             : AIC=inf, Time=5.89 sec
 ARIMA(2,1,0)(1,1,1)[7]             : AIC=inf, Time=5.76 sec
 ARIMA(3,1,0)(2,1,0)[7]             : AIC=4010.353, Time=0.77 sec
 ARIMA(2,1,1)(2,1,0)[7]             : AIC=4010.334, Time=0.76 s

In [74]:
train = ts.Close.loc['2010-01-01':'2018-01-01']
test_7 = ts.Close.loc['2017-12-20': '2018-01-01']
test_31 = ts.Close.loc['2017-11-15':'2018-01-01']
two_month = ts.Close.loc['2017-10-15':'2018-01-01']

model_7.fit(train)

future_forecast_7 = model_7.predict(n_periods=31)

In [77]:
future_forecast_7

array([76.78556528, 76.95566045, 76.76779684, 76.41432024, 76.43217199,
       76.20014895, 76.35340551, 76.18093729, 76.36154996, 76.12350829,
       75.74909828, 75.83155002, 75.65623178, 75.45199986, 75.00246445,
       75.24006554, 75.34608822, 75.12488364, 75.23350966, 75.22072247,
       75.25128111, 75.02728488, 75.22410744, 75.12426211, 74.81088814,
       74.8795801 , 74.74101681, 74.74577122, 74.47345324, 74.67656698,
       74.58848643])

In [112]:
print (test_31.values)

[75.93000031 75.97000122 75.94999695 76.88999939 77.23999786 77.16999817
 77.25       77.23000336 78.33999634 78.55999756 79.16000366 79.26000214
 78.93000031 78.51000214 78.31999969 77.79000092 78.77999878 79.22000122
 79.26999664 78.20999908 77.47000122 77.51000214 76.97000122 76.11000061
 75.05000305 75.40000153 76.08999634 76.12999725 76.33999634 76.66999817
 76.83000183]


In [114]:
fig = px.line(test_31, x=test_31.index, y=test_31.values)
fig.show()

In [134]:
fig = px.line(future_forecast_7, x=test_31, y=future_forecast_7)
fig.show()

In [129]:
future_forecast_7.nonzero()

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
       dtype=int64),)